In [1]:
import os
import sys
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import itertools
import numpy as np
from yaml import safe_load

# Walletexplorer

In [2]:
page = requests.get("https://www.walletexplorer.com/")

In [3]:
soup = BeautifulSoup(page.text, 'html.parser')

In [14]:
services = [list(map(lambda x: x.get('href'), i.find_all("a"))) for i in soup.find_all("table")[0].find_all("li")]
df = pd.DataFrame(columns=["service", "address"])
l = len(list(itertools.chain(*services)))

# Add largest but unknown wallet
services.append("/wallet/00000014ea8b260f")

In [15]:
c = 0
for service in services:
    print(service)
    w = np.random.randint(3,10)
    print("{}/{}".format(c, l))
    for version in service:
        if version.split("/")[-1] in set(df["service"]):
            continue
        print("https://www.walletexplorer.com{}/addresses".format(version))
        page = requests.get("https://www.walletexplorer.com{}/addresses".format(version))
        page = BeautifulSoup(page.text, 'html.parser')
        max_site = page.find_all("div", {"class": "paging"})[0].get_text()
        max_site = re.search("[0-9]+./.([0-9]+)", max_site).group().split(" / ")[1]
        url = "https://www.walletexplorer.com{}/addresses?page={}".format(version, max_site)
        print("Processing {}".format(url))
        page = requests.get("https://www.walletexplorer.com{}/addresses?page={}".format(version, max_site))
        page = BeautifulSoup(page.text, 'html.parser')
        addr = [i.get_text() for i in page.find_all("table")[0].find_all("a")]
        serv = [version.split("/")[-1]]*len(addr)
        addr = list(zip(serv, addr))
        addr = pd.DataFrame(addr, columns=["service", "address"])
        df = df.append(addr)
        time.sleep(w)
    c += 1
df.to_csv("services_walletexplorer.csv", index=False)

['/wallet/00000014ea8b260f']
0/388
https://www.walletexplorer.com/wallet/00000014ea8b260f/addresses
Processing https://www.walletexplorer.com/wallet/00000014ea8b260f/addresses?page=311659


# Graphsense

In [ ]:
df = pd.DataFrame(columns=["service", "address"])
for file in os.listdir("./graphsense-tagpacks/packs/address"):
    _label = None
    _df    = None
    if file in ["samourai.yaml", "wasabi_collector.yaml", "ofac.yaml"]:
        continue
    print("Parsing {}".format(file), end="\r")
    with open('./graphsense-tagpacks/packs/address/{}'.format(file), 'r') as f:
        _df    = pd.json_normalize(safe_load(f))
        try:
            _label = _df["label"][0]            
        except KeyError:
            pass
        try:
            if _df["currency"][0] != "BTC":
                continue
        except:
            pass
        
        for i in _df["tags"][0]:
            try:
                if i["currency"] != "BTC":
                    continue
            except:
                pass
            try:
                _label = i["label"]
            except:
                pass

            df.loc[len(df)] = [_label, i["address"]]
    
    print("Successfully parsed {:<28} {}: {:,.0f}".format(file+";", "Length of df", len(df)))

df.to_csv("./services_graphsense.csv", index=False)

# Merge both

In [52]:
df_we  = pd.read_csv('./services_walletexplorer.csv')
df_gs  = pd.read_csv('./services_graphsense.csv')
df_all = pd.concat([df_gs,df_we]).drop_duplicates(['address'], keep='last', ignore_index=True)
df_all.to_csv("./services_all.csv", index=False)